# FINM HW3

## Part 1

## Part 2

### 1. The Factors 

Calculate their univariate performance statistics:
* Mean
* Volatility
* Sharpe
* VaR

### 2. 
The factors are constructed in such a way as to reduce correlation between them.
Report the correlation matrix across the three factors. Does the construction method succeed
in keeping correlations small?

### 3. 
Plot the cumulative returns of the factors.

### 4. Test assets.
The “assets” tab provides monthly excess return data on various industry stock-portfolios.
Denote these as ri, for n = 1,...,12. Calculate the (annualized) univariate statistics from 1.1

### 5. 
Can the difference in mean excess returns of the portfolios be explained by differences in their
volatilities? Or by their VaR(.05) statistics?


## Problem 3

### 1.
For each of the n = 12 test assets, run the CAPM time-series regression

### 2.
Report the estimated βi,mkt, Treynor Ratio, αi, and Information Ratio for each of the n regressions.

### 3.
If (CAPM) were true, what would be true of the Treynor Ratios, alphas, and Information Ratios?

### 4.
Calculate the mean-absolute-error of the estimated alphas.

If the pricing model worked, should these alpha estimates be large or small? Why? Based on
your MAE stat, does this seem to support the pricing model or not?

## Problem 4

### 1.
For each regression, report the estimated α and r-squared.

### 2.
Calculate the mean-absolute-error of the estimated alphas.

If the pricing model worked, should these alpha estimates be large or small? Why? Based on
your MAE stat, does this seem to support the pricing model or not?


## Problem 5 (Extensions)